In [1]:
# Prepare corpus
data = open('all_lyrics.txt', encoding="utf8").read()
data = data.replace("<start>", "")
data = data.replace("<end>", "")
data = data.replace("<start> ", "")
data = data.replace(" <end>", "")
corpus = data.lower().split("\n")
print(corpus[:10])

[' aah ko chaahiye ik umra asar honay tak ', ' kaun jeeta hai teri ', ' kaun jeeta hai teri zulf ke sar honay tak ', ' kaun jeeta hai ', ' kaun jeeta hai teri zulf ke sar honay tak ', ' haaaan kaun jeeta hai teri ', ' aashiqui sabr talab haaye ', ' aur tamannaa betaab ', ' aashiqui sabr talab aur tamannaa betaab ', ' dil kaa kyaa rang karoon khoonejigar honay tak ']


In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
token = Tokenizer()
token.fit_on_texts(corpus)

In [4]:
encoded_text = token.texts_to_sequences(corpus)
# vocabulary size should be + 1
vocab_size = len(token.word_counts) + 1
print(vocab_size)

69228


In [5]:
datalist = []
for d in encoded_text:
    if len(d)>1:
        for i in range(2, len(d)):
            datalist.append(d[:i])

In [6]:
max_length = 30
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
print(seq_length)

29


In [11]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

NotImplementedError: Cannot convert a symbolic Tensor (lstm_3/strided_slice:0) to a numpy array.

In [ ]:
model.fit(X, y, batch_size=32, epochs=1000)

In [ ]:
poetry_length = 10
def generate_poetry(seed_text, n_lines):
    poem = ""
    for i in range(n_lines):
        text = []
        for _ in range(poetry_length):
            encoded = token.texts_to_sequences([seed_text])
            encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')
            y_pred = np.argmax(model.predict(encoded), axis=-1)

            predicted_word = ""
            for word, index in token.word_index.items():
                if index == y_pred:
                    predicted_word = word
                    break

            seed_text = seed_text + ' ' + predicted_word
            text.append(predicted_word)

        seed_text = text[-1]
        text = ' '.join(text)
        print(text)
        poem += text + "\n"
    print(poem)

In [ ]:
generate_poetry('pal bhar', 2)